# Machine Learning Model Building Pipeline: Feature Engineering

We will engineer the variales so that we tackle:

1. Missing values
2. Temporal variables
3. Non-Gaussigan distributed variables
4. Categorical variables: remove rare labels
5. Categorical variables: convert strings to numbers
6. Standarise the values of the variables to the same range

## Setting the seed

We are engineering varialbes and pre-processing data with the idea of deployin the mode. Each step includes some element of randomness, it is extremely importat that we set the seed. We can otain reproducibility between our research and our development code. Always set the seeds.

## Add imports

In [2]:
# to handle datasets
import pandas as pd 
import numpy as np 

# for plotting
import matplotlib.pyplot as plt

# to divide train and test set
from sklearn.model_selection import train_test_split

# feature scaling
from sklearn.preprocessing import MinMaxScaler

# to visualise all the columns in the dataframe
pd.pandas.set_option("display.max_columns", None)

import warnings
warnings.simplefilter(action='ignore')

## Load dataset

In [ ]:
data = pd.read_csv('house')